# 강남 3구는 안전한가?

In [63]:
import numpy as np
import pandas as pd
import os
os.chdir('C:/Users/HK/Desktop/HK/데이터사이언스/GitHub/python-data-handling')

## 1. 데이터 정리

In [64]:
crime_anal_police = pd.read_csv('data/02. crime_in_Seoul.csv', encoding = 'euc-kr')
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거
0,중부서,2,2,3,2,105,65,"1,395",477,"1,355","1,170"
1,종로서,3,3,6,5,115,98,"1,070",413,"1,278","1,070"
2,남대문서,1,0,6,4,65,46,"1,153",382,869,794
3,서대문서,2,2,5,4,154,124,"1,812",738,"2,056","1,711"
4,혜화서,3,2,5,4,96,63,"1,114",424,"1,015",861


* **Google Maps**로 경찰서의 위치 정보 받아오기

In [65]:
import googlemaps

In [66]:
station_name = []

for name in crime_anal_police['관서명']:
    station_name.append('서울' + str(name[:-1]) + '경찰서')

In [67]:
# (API 대체)
station_gmaps = pd.read_csv('data/02. station_gmaps_fix.csv')
station_gmaps.head()

,address,lat,long
0,대한민국 서울특별시 중구 을지로동 수표로 27,37.563647,126.989580
1,대한민국 서울특별시 종로구 종로1.2.3.4가동 율곡로 46,37.575558,126.984867
2,대한민국 서울특별시 중구 남대문로5가 한강대로 410,37.554758,126.973498
3,대한민국 서울특별시 서대문구 미근동 통일로 113,37.564785,126.966776
4,대한민국 서울특별시 종로구 종로1.2.3.4가동 창경궁로 112-16,37.571840,126.998856


In [68]:
station_address = [addr for addr in station_gmaps.address]
station_lat = [lat for lat in station_gmaps.lat]
station_lng = [long for long in station_gmaps.long]

In [69]:
gu_name = []

for name in station_address:
    tmp = name.split()
#     tmp_gu = [gu for gu in tmp if g[-1] == '구'][0]
    tmp_gu = tmp[2]
    gu_name.append(tmp_gu)

crime_anal_police['구별'] = gu_name
crime_anal_police.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",중구
1,종로서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",종로구
2,남대문서,1,0,6,4,65,46,"1,153",382,869,794,중구
3,서대문서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",서대문구
4,혜화서,3,2,5,4,96,63,"1,114",424,"1,015",861,종로구


In [70]:
crime_anal_police[crime_anal_police['관서명'] == '금천서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
15,금천서,3,4,6,6,151,122,"1,567",888,"2,054","1,776",관악구


In [108]:
crime_anal_raw = crime_anal_police.copy()

crime_anal_raw.loc[crime_anal_raw['관서명'] == '금천서', ['구별']] = '금천구'
crime_anal_raw[crime_anal_raw['관서명'] == '금천서']

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
15,금천서,3,4,6,6,151,122,"1,567",888,"2,054","1,776",금천구


    → 데이터 직접 수정

## 2. 범죄 데이터 구별 정리
`pd.pivot_table`

* **구별 범죄건수 Sum**

In [109]:
crime_anal_raw.head()

,관서명,살인 발생,살인 검거,강도 발생,강도 검거,강간 발생,강간 검거,절도 발생,절도 검거,폭력 발생,폭력 검거,구별
0,중부서,2,2,3,2,105,65,"1,395",477,"1,355","1,170",중구
1,종로서,3,3,6,5,115,98,"1,070",413,"1,278","1,070",종로구
2,남대문서,1,0,6,4,65,46,"1,153",382,869,794,중구
3,서대문서,2,2,5,4,154,124,"1,812",738,"2,056","1,711",서대문구
4,혜화서,3,2,5,4,96,63,"1,114",424,"1,015",861,종로구


In [110]:
crime_anal_raw.set_index('관서명', inplace = True)

crime_anal = pd.pivot_table(crime_anal_raw, index = '구별', aggfunc = np.sum)
crime_anal.head()

,강간 검거,강간 발생,강도 검거,강도 발생,살인 검거,살인 발생
구별,,,,,,
강남구,349,449,18,21,10,13
강동구,123,156,8,6,3,4
강북구,126,153,13,14,8,7
관악구,221,320,14,12,8,9
광진구,220,240,26,14,4,4


In [111]:
### 왜 절도 폭력은 빠지니!!
crime_anal_raw.apply(lambda x: np.sum(pd.isnull(x)))

살인 발생    0
살인 검거    0
강도 발생    0
강도 검거    0
강간 발생    0
강간 검거    0
절도 발생    0
절도 검거    0
폭력 발생    0
폭력 검거    0
구별       0
dtype: int64

**Ref.** 아래 R 코드와 같은 output
```
crime_anal %>% group_by(`구별`) %>% summarize_all(sum)
```

In [112]:
for i in range(0, 5, 2):
    arrest = crime_anal.columns[i]
    occur = crime_anal.columns[i + 1]
    per = str(crime_anal.columns[i].split()[0] + '검거율')
    
    crime_anal[per] = crime_anal[arrest] / crime_anal[occur] * 100

crime_anal.drop(columns = [crime_anal.columns[i] for i in range(0, 5, 2)], 
                inplace = True)
crime_anal.head()

,강간 발생,강도 발생,살인 발생,강간검거율,강도검거율,살인검거율
구별,,,,,,
강남구,449,21,13,77.728285,85.714286,76.923077
강동구,156,6,4,78.846154,133.333333,75.000000
강북구,153,14,7,82.352941,92.857143,114.285714
관악구,320,12,9,69.062500,116.666667,88.888889
광진구,240,14,4,91.666667,185.714286,100.000000
